In [49]:
import numpy as np
import pandas as pd
import dask
from importlib import reload

import pysam
from pysam import FastaFile, FastxFile
from pysam import AlignmentFile
from Bio import Restriction
from Bio.Seq import Seq
import pyranges as pr
import networkx as nx

from enum import Enum
from typing import Dict, List, NewType, Optional
from intake.source.base import DataSource, Schema
import pyarrow as pa
from pydantic import BaseModel, confloat, conint, constr

import post_align
import post_align_utils as putil

In [58]:
af = AlignmentFile("./NlaIII_run02_batch2_GRCm39.coord_sort.bam")
dir(af)

af.has_index()

False

1

In [2]:
reload(post_align)

path = "./NlaIII_run02_batch2_GRCm39.coord_sort.bam"
df = post_align.create_table(path)
print(df.shape)

(85437, 17)


In [3]:
reload(post_align)

path = 'GCF_000001635.27_GRCm39_genomic.fna' 
fdf = post_align.create_virtual_digest(path, digest_param='NlaIII')
print(fdf.shape)

(12876595, 5)


In [104]:
df.head()

,read_idx,align_idx,align_type,chrom,start,end,strand,read_name,read_length,read_start,read_end,mapping_quality,align_score,align_base_qscore,phase_set,phase_qual,haplotype
15246,8034,61068,primary,NC_000068.8,146367157,146367382,False,00051e92-51de-4694-a986-83654849a5e5,1496,694,907,16,112,10,0,0,-1
15159,8034,61067,primary,NC_000068.8,144167507,144167702,False,00051e92-51de-4694-a986-83654849a5e5,1496,490,686,39,169,16,0,0,-1
15185,8034,61065,primary,NC_000068.8,144168062,144168528,True,00051e92-51de-4694-a986-83654849a5e5,1496,94,556,47,388,14,0,0,-1
15068,8034,61066,primary,NC_000068.8,141740110,141740574,True,00051e92-51de-4694-a986-83654849a5e5,1496,1002,1439,219,263,10,0,0,-1
85107,4442,33849,primary,NW_023337853.1,13495,13583,False,00055210-4698-4dbe-9fe6-28ee7cf02387,5651,4985,5075,8,84,13,0,0,-1


In [4]:
tmp = df.groupby('read_idx', as_index=False).agg({
    'align_idx':['count', 'nunique']
})

print(tmp.shape)

mask = (tmp['align_idx']['count'] != tmp['align_idx']['nunique'])

tmp = tmp[mask]
print(tmp.shape)

(11176, 3)
(0, 3)


In [117]:
fdf.head()

,start,end,chrom,fragment_length,fragment_id
0,0,3050049,NC_000067.7,3050049,1
1,3050049,3050054,NC_000067.7,5,2
2,3050054,3050842,NC_000067.7,788,3
3,3050842,3050905,NC_000067.7,63,4
4,3050905,3050945,NC_000067.7,40,5


In [13]:
df.columns

Index(['read_idx', 'align_idx', 'align_type', 'chrom', 'start', 'end',
       'strand', 'read_name', 'read_length', 'read_start', 'read_end',
       'mapping_quality', 'align_score', 'align_base_qscore', 'phase_set',
       'phase_qual', 'haplotype'],
      dtype='object')

In [99]:
fdf.columns

Index(['start', 'end', 'chrom', 'fragment_length', 'fragment_id'], dtype='object')

In [35]:
reload(putil)

<module 'post_align_utils' from '/home/cstansbu/git_repositories/single_cell_poreC/development/post_align_utils.py'>

In [50]:
pore_c = assign_fragments(df, fdf)
print(pore_c.shape)
pore_c.head()

(85437, 30)


,read_idx,align_idx,align_type,chrom,start,end,strand,read_name,read_length,read_start,...,num_contained_fragments,num_overlapping_fragments,overlap_length,fragment_start,fragment_end,perc_of_alignment,perc_of_fragment,is_contained,align_start,align_end
48467,0,0,primary,NC_000082.7,39409284,39409846,False,916c6a09-32f4-4174-bf48-fa970e6b8329,1443,271,...,2,4,372,39409456,39409828,66.192169,100.000000,True,39409284,39409846
48468,0,1,primary,NC_000082.7,39207565,39208106,False,916c6a09-32f4-4174-bf48-fa970e6b8329,1443,821,...,0,2,537,39207569,39208525,99.260628,56.171547,False,39207565,39208106
48466,0,2,primary,NC_000082.7,39406511,39406676,False,916c6a09-32f4-4174-bf48-fa970e6b8329,1443,57,...,1,2,128,39406426,39406639,77.575760,60.093895,False,39406511,39406676
47660,1,3,primary,NC_000069.7,47002317,47003214,True,8eaf88d8-6d21-4643-9df9-592fd00e134d,5414,3813,...,0,2,893,47002321,47003770,99.554070,61.628708,False,47002317,47003214
47661,1,4,primary,NC_000069.7,47002317,47003214,True,8eaf88d8-6d21-4643-9df9-592fd00e134d,5414,2131,...,0,2,893,47002321,47003770,99.554070,61.628708,False,47002317,47003214
